<a href="https://colab.research.google.com/github/HesusG/diagnostico-lineas-accion/blob/main/Semana2/ejercicios/workshop2_plantilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop 2: Pruebas Estadísticas Avanzadas

**Curso:** CD2001B - Diagnóstico para Líneas de Acción  
**Módulo:** 1 - Análisis Estadístico  
**Valor:** 10% de la calificación final  
**Fecha límite:** [Definir según calendario]

---

## 📋 Instrucciones Generales

Este workshop evalúa tu capacidad para:
- Realizar pruebas de Ji-cuadrada para variables categóricas
- Ejecutar ANOVA para comparar múltiples grupos
- Crear y evaluar modelos de regresión lineal simple
- Interpretar resultados estadísticos en contexto educativo

**Dataset:** `student-alcohol-consumption.csv`

**Formato de entrega:**
- Este notebook completado (.ipynb)
- Todas las celdas ejecutadas
- Interpretaciones completas en español

---

## 🎯 Rúbrica de Evaluación

| Criterio | Puntos | Descripción |
|----------|--------|-------------|
| Parte 1: Prueba Ji-Cuadrada | 25 | Tabla contingencia, ejecución, interpretación |
| Parte 2: ANOVA | 35 | Grupos correctos, supuestos, post-hoc, interpretación |
| Parte 3: Regresión Lineal | 40 | Modelo, evaluación, predicciones, interpretación |
| **TOTAL** | **100** | |

---

## 0. Información del Estudiante

**Nombre completo:** [Escribe tu nombre aquí]  
**Matrícula:** [Tu matrícula]  
**Fecha de entrega:** [Fecha]  

---

## 1. Preparación del Entorno

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
%matplotlib inline

print("✓ Librerías importadas correctamente")

In [ ]:
# Cargar dataset
df = pd.read_csv('https://raw.githubusercontent.com/HesusG/diagnostico-lineas-accion/main/Semana1/datos/student-alcohol-consumption.csv')

print(f"Dataset cargado: {df.shape[0]} estudiantes, {df.shape[1]} variables")
print(f"\nPrimeras filas:")
df.head()

---

## 📝 RECORDATORIO: Reflexiones Académicas con Estructura MEAL

### 🔗 Conexión con Workshop 1

En el **Workshop 1** aprendiste la estructura **MEAL** para escribir reflexiones académicas. Este Workshop 2 **construye sobre lo aprendido** en la Semana 1:

| Workshop 1 (Semana 1) | Workshop 2 (Semana 2) |
|----------------------|----------------------|
| Estadística descriptiva | Chi-cuadrada (variables categóricas) |
| Prueba t (una muestra) | ANOVA (múltiples grupos) |
| Prueba t (dos muestras) | Regresión lineal (predicción) |
| **Mismo dataset:** `student-alcohol-consumption.csv` | **Mismo dataset:** Continuidad analítica |

---

### 📊 Recordatorio Rápido: MEAL

| Componente | Qué incluir | Longitud |
|------------|-------------|----------|
| **M - Main Idea** | Tu tesis o argumento principal | 50-75 palabras |
| **E - Evidence** | Datos específicos de TUS análisis (p-values, estadísticos, medias) | 100-125 palabras |
| **A - Analysis** | Interpretación con referencias académicas (APA 7) | 100-125 palabras |
| **L - Link** | Implicaciones prácticas y conexión con ambos workshops | 50-75 palabras |

**Total esperado:** 300-400 palabras

---

### 💡 Novedad en Workshop 2: Análisis Comparativo

Tu reflexión final en este workshop debe **integrar hallazgos de AMBAS semanas**:

✅ **Compara técnicas:** ¿Qué preguntas respondió cada método?  
✅ **Integra resultados:** ¿Cómo se complementan los hallazgos de ambos workshops?  
✅ **Sintetiza aprendizajes:** ¿Qué entiendes ahora que no entendías en Semana 1?  

**Ejemplo de integración:**
> "En Workshop 1, la prueba t reveló diferencias en G3 por consumo de alcohol (p = 0.032). En Workshop 2, ANOVA mostró que el tiempo de estudio también influye (F = 4.52, p = 0.004). La regresión identificó que ausencias predicen G3 (R² = 0.15). **Juntos, estos análisis sugieren que el rendimiento académico es multifactorial** (Richardson et al., 2012)."

---

### 📚 Referencias y Plantillas

- **Lista de referencias:** Encontrarás las mismas fuentes académicas sugeridas al final de este notebook
- **Plantillas de oraciones:** Usa los sentence starters del Workshop 1 si los necesitas
- **Rúbrica:** 10 puntos evaluados con los mismos criterios

**Consejo:** Revisa tu reflexión del Workshop 1 antes de escribir esta. La calidad mejora cuando ves tu propio progreso.

---


---

## PARTE 1: Prueba Ji-Cuadrada (χ²) de Independencia (25 puntos)

### Contexto

Queremos investigar si existe una relación entre el **género** de los estudiantes (`sex`) y su **aspiración de continuar con educación superior** (`higher`).

**Variables:**
- `sex`: Género del estudiante (F = Femenino, M = Masculino)
- `higher`: ¿Desea cursar educación superior? (yes, no)

---

### 1.1 Planteamiento de Hipótesis (5 puntos)

**Pregunta 1.1:** Plantea las hipótesis nula y alternativa para esta prueba.

**Tu respuesta:**

- **H₀:** [Escribe la hipótesis nula aquí - ¿Las variables son independientes?]
- **H₁:** [Escribe la hipótesis alternativa aquí]
- **Nivel de significancia (α):** 0.05

---

### 1.2 Tabla de Contingencia (5 puntos)

Crea una tabla de contingencia que muestre la relación entre `sex` y `higher`.

In [ ]:
# TU CÓDIGO AQUÍ
# Crear tabla de contingencia

tabla = # COMPLETA (usa pd.crosstab)

print("Tabla de Contingencia: Género vs Aspiración Educación Superior")
print("="*60)
print(tabla)
print(f"\nTotal de estudiantes: {tabla.sum().sum()}")

In [ ]:
# Visualizar la tabla con heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(tabla, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Frecuencia'})
plt.title('Relación: Género vs Aspiración Educación Superior', fontsize=14, fontweight='bold')
plt.xlabel('Educación Superior', fontsize=12)
plt.ylabel('Género', fontsize=12)
plt.tight_layout()
plt.show()

### 1.3 Ejecución de la Prueba (10 puntos)

Realiza la prueba de Ji-cuadrada.

In [ ]:
# TU CÓDIGO AQUÍ
# Ejecutar prueba de Ji-cuadrada

chi2, p_value, dof, expected = # COMPLETA (usa chi2_contingency)

print("="*60)
print("RESULTADOS DE LA PRUEBA JI-CUADRADA")
print("="*60)
print(f"Estadístico χ²: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Grados de libertad: {dof}")
print(f"Nivel de significancia (α): 0.05")
print("="*60)

# Mostrar frecuencias esperadas
print("\nFrecuencias Esperadas (bajo H₀):")
expected_df = pd.DataFrame(expected, index=tabla.index, columns=tabla.columns)
print(expected_df.round(2))

### 1.4 Interpretación (5 puntos)

**Pregunta 1.4a:** ¿Cuál es tu decisión estadística? ¿Rechazas H₀?

**Tu respuesta:**  
[Escribe tu decisión aquí]

**Pregunta 1.4b:** ¿Qué significa este resultado? ¿El género está relacionado con la aspiración de educación superior en esta muestra?

**Tu respuesta:**  
[Escribe tu interpretación aquí]

---

---

## PARTE 2: ANOVA (Análisis de Varianza) (35 puntos)

### Contexto

Queremos investigar si el **tiempo de estudio semanal** (`studytime`) tiene relación con las **calificaciones finales** (`G3`).

**Variable:**
- `studytime`: Tiempo de estudio semanal
  - 1: < 2 horas
  - 2: 2-5 horas
  - 3: 5-10 horas
  - 4: > 10 horas
- `G3`: Calificación final (0-20)

---

### 2.1 Planteamiento de Hipótesis (5 puntos)

**Pregunta 2.1:** Plantea las hipótesis para el ANOVA.

**Tu respuesta:**

- **H₀:** [Las medias de calificación son iguales entre todos los grupos de tiempo de estudio]
- **H₁:** [Al menos una media es diferente]
- **Nivel de significancia (α):** 0.05

---

### 2.2 Exploración de Grupos (5 puntos)

Calcula estadísticas descriptivas de `G3` para cada nivel de `studytime`.

In [ ]:
# TU CÓDIGO AQUÍ
# Estadísticas por grupo

estadisticas = df.groupby('studytime')['G3'].agg([
    ('n', 'count'),
    ('Media', 'mean'),
    ('Mediana', 'median'),
    ('Desv.Std', 'std')
]).round(2)

print("Estadísticas de G3 por Tiempo de Estudio:")
print("="*60)
print(estadisticas)

In [ ]:
# Visualización con boxplots
plt.figure(figsize=(10, 6))
df.boxplot(column='G3', by='studytime', patch_artist=True, grid=False)
plt.suptitle('')  # Eliminar título automático
plt.title('Comparación de Calificaciones por Tiempo de Estudio', fontsize=14, fontweight='bold')
plt.xlabel('Tiempo de Estudio Semanal (1:<2h, 2:2-5h, 3:5-10h, 4:>10h)', fontsize=11)
plt.ylabel('Calificación Final (G3)', fontsize=12)
plt.tight_layout()
plt.show()

**Pregunta 2.2:** Observando el boxplot y las estadísticas, ¿qué tendencia preliminar notas?

**Tu respuesta:**  
[Escribe tu observación aquí]

---

### 2.3 Ejecución del ANOVA (10 puntos)

Realiza el análisis de varianza.

In [ ]:
# TU CÓDIGO AQUÍ
# Separar grupos

grupo1 = df[df['studytime'] == 1]['G3']
grupo2 = df[df['studytime'] == 2]['G3']
grupo3 = df[df['studytime'] == 3]['G3']
grupo4 = df[df['studytime'] == 4]['G3']

# Ejecutar ANOVA
f_stat, p_value_anova = # COMPLETA (usa stats.f_oneway)

print("="*60)
print("RESULTADOS DEL ANOVA")
print("="*60)
print(f"Estadístico F: {f_stat:.4f}")
print(f"p-value: {p_value_anova:.4f}")
print(f"Nivel de significancia (α): 0.05")
print("="*60)

### 2.4 Prueba Post-Hoc (Tukey) (10 puntos)

Si rechazaste H₀, realiza una prueba post-hoc para identificar qué grupos son diferentes.

In [ ]:
# TU CÓDIGO AQUÍ
# Prueba post-hoc de Tukey

from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Ejecutar Tukey HSD
tukey = pairwise_tukeyhsd(endog=df['G3'], groups=df['studytime'], alpha=0.05)

print("Resultados de Prueba Post-Hoc (Tukey HSD):")
print("="*70)
print(tukey)
print("="*70)

### 2.5 Interpretación (5 puntos)

**Pregunta 2.5a:** ¿Qué concluyes del ANOVA? ¿Hay diferencias significativas?

**Tu respuesta:**  
[Escribe tu conclusión aquí]

**Pregunta 2.5b:** Según la prueba de Tukey, ¿qué grupos específicos son significativamente diferentes? ¿Qué implica esto en términos educativos?

**Tu respuesta:**  
[Escribe tu interpretación detallada aquí]

---

---

## PARTE 3: Regresión Lineal Simple (40 puntos)

### Contexto

Queremos crear un modelo que **prediga la calificación final** (`G3`) basándose en el **número de ausencias** (`absences`).

**Variables:**
- Variable independiente (X): `absences` (número de ausencias)
- Variable dependiente (Y): `G3` (calificación final)

---

### 3.1 Análisis Exploratorio (5 puntos)

Crea un scatter plot de ausencias vs calificación.

In [ ]:
# TU CÓDIGO AQUÍ
# Scatter plot

plt.figure(figsize=(10, 6))
plt.scatter(df['absences'], df['G3'], alpha=0.5, edgecolors='k')
plt.title('Relación entre Ausencias y Calificación Final', fontsize=14, fontweight='bold')
plt.xlabel('Número de Ausencias', fontsize=12)
plt.ylabel('Calificación Final (G3)', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

**Pregunta 3.1:** ¿Observas alguna relación visual entre ausencias y calificaciones? ¿Parece ser lineal?

**Tu respuesta:**  
[Escribe tu observación aquí]

---

### 3.2 Correlación de Pearson (5 puntos)

Calcula el coeficiente de correlación entre `absences` y `G3`.

In [ ]:
# TU CÓDIGO AQUÍ
# Calcular correlación

correlacion, p_value_corr = # COMPLETA (usa stats.pearsonr)

print("="*60)
print("CORRELACIÓN DE PEARSON")
print("="*60)
print(f"Coeficiente de correlación (r): {correlacion:.4f}")
print(f"p-value: {p_value_corr:.4f}")
print("="*60)

**Pregunta 3.2:** Interpreta el valor de correlación. ¿Es fuerte, moderada o débil? ¿Es positiva o negativa?

**Tu respuesta:**  
[Escribe tu interpretación aquí]

---

### 3.3 Creación del Modelo de Regresión (10 puntos)

Ajusta un modelo de regresión lineal simple.

In [ ]:
# TU CÓDIGO AQUÍ
# Preparar datos

X = df['absences']
y = df['G3']

# Añadir constante (intercepto)
X_const = sm.add_constant(X)

# Ajustar modelo
modelo = # COMPLETA (usa sm.OLS y .fit())

# Mostrar resumen
print(modelo.summary())

### 3.4 Extracción de Parámetros (5 puntos)

Extrae e interpreta los coeficientes del modelo.

In [ ]:
# TU CÓDIGO AQUÍ
# Extraer coeficientes

intercepto = modelo.params['const']
pendiente = modelo.params['absences']
r_cuadrado = modelo.rsquared

print("="*60)
print("PARÁMETROS DEL MODELO")
print("="*60)
print(f"Intercepto (β₀): {intercepto:.4f}")
print(f"Pendiente (β₁): {pendiente:.4f}")
print(f"R² (R-cuadrado): {r_cuadrado:.4f}")
print("="*60)
print(f"\nEcuación del modelo:")
print(f"G3 = {intercepto:.2f} + ({pendiente:.4f}) × absences")
print("="*60)

**Pregunta 3.4:** Interpreta los coeficientes:

- **Intercepto:** ¿Qué representa en este contexto?
- **Pendiente:** ¿Qué significa que sea positiva o negativa? ¿En cuánto cambia la calificación por cada ausencia adicional?
- **R²:** ¿Qué porcentaje de la variabilidad en calificaciones explica el modelo?

**Tu respuesta:**  
[Escribe tus interpretaciones aquí]

---

### 3.5 Visualización del Modelo (5 puntos)

Grafica los datos con la línea de regresión ajustada.

In [ ]:
# TU CÓDIGO AQUÍ
# Gráfico con línea de regresión

plt.figure(figsize=(10, 6))
plt.scatter(df['absences'], df['G3'], alpha=0.5, edgecolors='k', label='Datos observados')

# Línea de regresión
x_line = np.linspace(df['absences'].min(), df['absences'].max(), 100)
y_line = intercepto + pendiente * x_line
plt.plot(x_line, y_line, 'r-', linewidth=2, label=f'Regresión: G3 = {intercepto:.2f} + {pendiente:.4f}×absences')

plt.title('Modelo de Regresión Lineal: Ausencias vs Calificación', fontsize=14, fontweight='bold')
plt.xlabel('Número de Ausencias', fontsize=12)
plt.ylabel('Calificación Final (G3)', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 3.6 Predicciones (10 puntos)

Usa el modelo para hacer predicciones.

In [ ]:
# TU CÓDIGO AQUÍ
# Hacer predicciones para diferentes números de ausencias

ausencias_prediccion = [0, 5, 10, 20]

print("="*60)
print("PREDICCIONES DEL MODELO")
print("="*60)

for ausencias in ausencias_prediccion:
    # Predecir calificación
    calificacion_pred = # COMPLETA (usa la ecuación del modelo)
    print(f"Con {ausencias} ausencias → Calificación predicha: {calificacion_pred:.2f}")

print("="*60)

**Pregunta 3.6:** ¿Son razonables estas predicciones? ¿Hay algún valor que parezca poco realista? ¿Por qué?

**Tu respuesta:**  
[Escribe tu análisis aquí]

---

---

## PARTE 4: Reflexión Académica Comparativa (BONUS - 10 puntos)

### 📝 Estructura Requerida: MEAL Comparativo

Esta reflexión debe **integrar hallazgos de Workshop 1 Y Workshop 2**. Compara las técnicas aprendidas y sintetiza tus conclusiones.

---

### 📊 Requisitos de Contenido

| Componente | Requisitos | Puntos |
|------------|-----------|--------|
| **M - Main Idea** | Tesis que sintetiza hallazgos de AMBOS workshops (50-75 palabras) | 2.5 |
| **E - Evidence** | Datos específicos de Workshop 1 Y Workshop 2 (100-125 palabras) | 2.5 |
| **A - Analysis** | Interpretación con ≥2 citas APA 7 (100-125 palabras) | 2.5 |
| **L - Link** | Implicaciones educativas y aprendizajes integrados (50-75 palabras) | 2.5 |
| **TOTAL** | 300-400 palabras | **10 puntos** |

---

### 🎯 Preguntas Guía para la Reflexión Comparativa

Tu reflexión debe responder (integradas en estructura MEAL, no como lista):

1. **Comparación de técnicas:**
   - ¿Qué tipo de preguntas respondió cada método estadístico?
   - ¿Cuándo usarías prueba t vs ANOVA vs regresión vs chi-cuadrada?
   
2. **Integración de hallazgos:**
   - ¿Cómo se complementan los resultados de ambos workshops?
   - ¿Qué patrones consistentes encontraste sobre el rendimiento académico?
   
3. **Implicaciones prácticas:**
   - Si fueras director de una escuela, ¿qué políticas implementarías basándote en TODOS los análisis?
   - ¿Qué factores priorizarías: consumo de alcohol, tiempo de estudio, ausencias, aspiraciones educativas?
   
4. **Limitaciones y aprendizajes:**
   - ¿Qué limitaciones tienen estos análisis en conjunto?
   - ¿Qué aprendiste sobre el proceso de análisis estadístico completo?

---

### 📝 Plantilla de Estructura

```markdown
**[M - Main Idea]**
[Tu tesis integradora aquí - debe mencionar hallazgos de AMBOS workshops]

**[E - Evidence]**
[Datos específicos del Workshop 1: prueba t, descriptivos]
[Datos específicos del Workshop 2: chi-cuadrada, ANOVA, regresión]
[Comparación cuantitativa de hallazgos]

**[A - Analysis]**
[Interpretación de patrones encontrados]
[Conexión con literatura académica - cita 1 (Autor, año)]
[Análisis crítico - cita 2 (Autor, año)]
[Discusión de causalidad vs correlación]

**[L - Link]**
[Implicaciones para política educativa]
[Aprendizajes personales del proceso completo]
[Conexión con objetivos del curso]
```

---

### 💡 Sentence Starters para Comparación

**Para Main Idea Comparativa:**
- "El análisis estadístico de ambos workshops revela que..."
- "La integración de técnicas descriptivas (W1) y multivariadas (W2) demuestra..."
- "Los hallazgos consistentes a través de seis pruebas estadísticas sugieren..."

**Para Evidence Comparativa:**
- "En Workshop 1, la prueba t mostró... (t = X, p = Y), mientras que en Workshop 2..."
- "La progresión de análisis univariado (W1) a multivariado (W2) reveló..."
- "Comparando los tamaños de efecto: Workshop 1 (d = X) vs Workshop 2 (η² = Y)..."

**Para Analysis Comparativa:**
- "Estos hallazgos convergen con la literatura sobre... (Autor, año) y... (Autor, año)"
- "La consistencia entre métodos refuerza la validez de..."
- "Sin embargo, las limitaciones de correlación (W1 y W2) indican que..."

**Para Link Comparativo:**
- "Integrando ambos workshops, las instituciones educativas deberían..."
- "Este proceso de análisis completo me enseñó que..."
- "Los próximos pasos en investigación deberían considerar tanto... (W1) como... (W2)"

---

### ✅ Ejemplo Completo (398 palabras)

**[M - Main Idea]**
El análisis estadístico integrado de los Workshops 1 y 2 revela que el rendimiento académico (G3) de estudiantes portugueses es un fenómeno **multifactorial** influenciado significativamente por factores conductuales (consumo de alcohol, ausencias) y motivacionales (aspiraciones educativas, tiempo de estudio). Las seis pruebas realizadas—descriptivos, t-test, chi-cuadrada, ANOVA, regresión—convergen en una conclusión: **intervenciones educativas efectivas deben ser integrales**, no enfocadas en un solo factor.

**[E - Evidence]**
En Workshop 1, la prueba t de dos muestras demostró que estudiantes con bajo consumo de alcohol obtienen calificaciones significativamente mayores (M = 11.2, SD = 3.4) que aquellos con alto consumo (M = 9.1, SD = 4.1; t = 2.18, p = 0.032). Los análisis del Workshop 2 complementan esto: **ANOVA** reveló diferencias significativas en G3 según tiempo de estudio (F = 4.52, p = 0.004), con estudiantes que estudian >10 horas semanales superando por 2.3 puntos a quienes estudian <2 horas. La **regresión lineal** identificó que cada ausencia adicional predice una disminución de 0.14 puntos en G3 (β = -0.14, p = 0.006, R² = 0.15). Finalmente, **chi-cuadrada** mostró que la aspiración de educación superior no depende del género (χ² = 1.23, p = 0.267), sugiriendo equidad motivacional.

**[A - Analysis]**
La consistencia entre métodos refuerza la **validez convergente** de estos hallazgos (Richardson et al., 2012). Paschal et al. (2005) documentaron mecanismos causales similares entre consumo de alcohol y rendimiento académico, incluyendo deterioro cognitivo y reducción de tiempo de estudio. Sin embargo, los diseños transversales de ambos workshops limitan inferencias causales; como advierte Faggiano et al. (2008), correlación no implica causalidad. La varianza explicada moderada (R² = 0.15 en regresión) indica que **factores no medidos**—como calidad de enseñanza o estatus socioeconómico (Cortés & Conchado, 2012)—también contribuyen significativamente al rendimiento.

**[L - Link]**
Las instituciones educativas deben implementar **programas integrados** que aborden consumo de alcohol, gestión de tiempo, y políticas de asistencia simultáneamente, no intervenciones aisladas. Mi aprendizaje clave fue comprender que diferentes técnicas estadísticas responden preguntas complementarias: descriptivos exploran, t-tests comparan, ANOVA generaliza comparaciones, chi-cuadrada examina independencia, y regresión predice. Investigaciones futuras deberían emplear diseños **longitudinales** (Steinberg, 2005) para establecer temporalidad causal entre estos factores y rendimiento académico.

---

### 📝 ESCRIBE TU REFLEXIÓN AQUÍ

**[M - Main Idea]**

[Escribe tu tesis integradora aquí - 50-75 palabras]

---

**[E - Evidence]**

[Escribe tu evidencia de AMBOS workshops aquí - 100-125 palabras]

---

**[A - Analysis]**

[Escribe tu análisis con ≥2 citas APA 7 aquí - 100-125 palabras]

---

**[L - Link]**

[Escribe tus implicaciones y aprendizajes aquí - 50-75 palabras]

---


---

# 📚 LISTA DE REFERENCIAS SUGERIDAS

Usa estas referencias académicas para tu reflexión. Cítalas en formato APA 7.

---

## Sobre Alcohol y Rendimiento Académico

Paschal, M. J., Grube, J. W., & Kypri, K. (2005). Alcohol expectancies and problem drinking among New Zealand university students. *Journal of Studies on Alcohol, 66*(6), 789-795. https://doi.org/10.15288/jsa.2005.66.789

> **Cuándo citar:** Para discutir mecanismos causales entre consumo de alcohol y rendimiento académico.

---

## Sobre Predictores de Rendimiento Académico

Richardson, M., Abraham, C., & Bond, R. (2012). Psychological correlates of university students' academic performance: A systematic review and meta-analysis. *Psychological Bulletin, 138*(2), 353-387. https://doi.org/10.1037/a0026838

> **Cuándo citar:** Para contextualizar tus hallazgos con meta-análisis de factores académicos.

---

## Sobre Contexto Familiar y Educación

Cortés, A., & Conchado, A. (2012). Los contextos parentales y el rendimiento académico: Un estudio comparativo entre estudiantes de bachillerato. *Revista Española de Orientación y Psicopedagogía, 23*(1), 56-69. https://doi.org/10.5944/reop.vol.23.num.1.2012.11392

> **Cuándo citar:** Para discutir variables familiares (Medu, Fedu, famrel) en tus análisis.

---

## Sobre Desarrollo Adolescente

Steinberg, L. (2005). Cognitive and affective development in adolescence. *Trends in Cognitive Sciences, 9*(2), 69-74. https://doi.org/10.1016/j.tics.2004.12.005

> **Cuándo citar:** Para explicar comportamientos de riesgo (alcohol, ausencias) desde perspectiva del desarrollo.

---

## Sobre Intervenciones Escolares

Faggiano, F., Vigna-Taglianti, F. D., Versino, E., Zambon, A., Borraccino, A., & Lemma, P. (2008). School-based prevention for illicit drugs use: A systematic review. *Preventive Medicine, 46*(5), 385-396. https://doi.org/10.1016/j.ypmed.2007.11.012

> **Cuándo citar:** Para proponer intervenciones educativas basadas en evidencia.

---

## Sobre Tamaños de Efecto en Educación

Thompson, B. (2007). Effect sizes, confidence intervals, and confidence intervals for effect sizes. *Psychology in the Schools, 44*(5), 423-432. https://doi.org/10.1002/pits.20234

> **Cuándo citar:** Para discutir significancia práctica (no solo estadística) de tus hallazgos.

---

### 📝 Cómo Citar en Formato APA 7

**En el texto:**
- Primera mención: (Richardson, Abraham, & Bond, 2012)
- Menciones posteriores: (Richardson et al., 2012)
- Como parte de la oración: Richardson et al. (2012) encontraron que...

**En la lista de referencias:**
- Copia la referencia completa tal como aparece arriba
- Ordena alfabéticamente por apellido del primer autor
- Usa sangría francesa (segunda línea indentada)

---


---

# 📊 RÚBRICA: Reflexión Académica MEAL (10 puntos)

| Criterio | Excelente (100%) | Bueno (75%) | Suficiente (50%) | Insuficiente (0%) | Puntos |
|----------|------------------|-------------|------------------|-------------------|--------|
| **Estructura MEAL** (4 pts) | Las 4 secciones claramente identificadas, longitud adecuada, flujo lógico | 3 secciones completas, longitud cercana a lo esperado | 2 secciones identificables, desbalanceado | Sin estructura MEAL reconocible | __/4 |
| **Uso de Evidencia** (2 pts) | Datos específicos y precisos (estadísticos, p-values, medias) de TUS análisis | Datos presentes pero sin especificidad ("se encontró diferencia") | Evidencia genérica o no relacionada con tus resultados | Sin evidencia numérica | __/2 |
| **Citaciones APA 7** (2 pts) | ≥2 citas correctamente formateadas, bien integradas en el análisis | 1-2 citas con errores menores de formato | Citas presentes pero mal formateadas o fuera de lugar | Sin citas o plagio | __/2 |
| **Pensamiento Crítico** (2 pts) | Análisis profundo, discute limitaciones, conexiones con literatura, implicaciones | Análisis superficial pero coherente | Solo descripción, sin análisis real | Simplemente repite resultados | __/2 |
| **TOTAL** | | | | | **__/10** |

---

### 💡 Consejos para Maximizar tu Calificación

✅ **Para Estructura MEAL (4 pts):**
- Etiqueta claramente cada sección: **[M - Main Idea]**, **[E - Evidence]**, etc.
- Respeta las longitudes sugeridas (50-75, 100-125, 100-125, 50-75 palabras)
- Asegúrate de que cada sección cumpla su función específica

✅ **Para Uso de Evidencia (2 pts):**
- Incluye números exactos: "p = 0.032", no "p fue significativo"
- Reporta estadísticos completos: "(M = 11.2, SD = 3.4; t = 2.18, p = 0.032)"
- Usa tus propios resultados, no ejemplos genéricos

✅ **Para Citaciones APA 7 (2 pts):**
- Formato correcto: (Autor, año) o Autor (año)
- Integradas naturalmente en el texto, no como lista al final
- Usa la lista de referencias sugeridas arriba

✅ **Para Pensamiento Crítico (2 pts):**
- Discute limitaciones: "Sin embargo, la correlación no implica..."
- Conecta con literatura: "Estos hallazgos son consistentes con..."
- Propone implicaciones prácticas: "Las instituciones deberían..."

---


---

## ✅ Checklist Antes de Entregar

Antes de enviar tu workshop, verifica:

- [ ] Todas las celdas están ejecutadas con resultados visibles
- [ ] Completaste TODOS los espacios "TU CÓDIGO AQUÍ"
- [ ] Respondiste TODAS las preguntas de interpretación
- [ ] Tus gráficos son claros y tienen títulos/etiquetas
- [ ] Verificaste que las pruebas estadísticas dan resultados coherentes
- [ ] Interpretaste los p-values correctamente
- [ ] Revisaste ortografía y redacción
- [ ] El notebook se ejecuta sin errores de principio a fin
- [ ] Incluiste tu nombre y matrícula al inicio

**Formato de nombre de archivo:**  
`Workshop2_NombreApellido_Matricula.ipynb`

---

**¡Excelente trabajo completando el Módulo 1!** 🎓📊🎉

---